In [1]:
# cd /content/drive/MyDrive/DATASET


/content/drive/MyDrive/DATASET


In [2]:
ls

feature_data/  MMTV2.pth  test1/                    text_hc_clustered.csv   text_hc_models/
MMTV1.pth      MMTV3.pth  text_deep_features_2.csv  text_hc_features_2.csv  videodeepmodel/


run this

In [1]:
import pandas as pd
import numpy as np
# Load text datasets
text_deep = pd.read_csv("/kaggle/input/fi-v2-test-val-data/FI V2 COMPLETE FEATURE DATASET/DEEP/text_deep_features.csv")
text_hc = pd.read_csv("/kaggle/input/fi-v2-test-val-data/FI V2 COMPLETE FEATURE DATASET/HANDCRAFTED/text_hc_features.csv")

# Expand text data by repeating each row 5 times (to match 5 segments per video)
text_deep_expanded = text_deep.loc[text_deep.index.repeat(5)].reset_index(drop=True)
text_hc_expanded = text_hc.loc[text_hc.index.repeat(5)].reset_index(drop=True)

# Add Segment_ID (assign values cyclically: [0,1,2,3,4])
text_deep_expanded["Segment_ID"] = np.tile(np.arange(5), len(text_deep))
text_hc_expanded["Segment_ID"] = np.tile(np.arange(5), len(text_hc))


# Save the processed files (optional)
text_deep_expanded.to_csv("/kaggle/working/text_deep_features_2.csv", index=False)
text_hc_expanded.to_csv("/kaggle/working/text_hc_features_2.csv", index=False)

# Print new shapes for confirmation
print("✅ Text Deep Aligned Shape:", text_deep_expanded.shape)  # Should be ~30,000 rows
print("✅ Text HC Aligned Shape:", text_hc_expanded.shape)      # Should be ~30,000 rows


✅ Text Deep Aligned Shape: (50000, 1547)
✅ Text HC Aligned Shape: (50000, 139)


## LIBRARIES

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.optim as optim


## File paths


In [4]:
file_paths = {
    "audio_deep": "/kaggle/input/fi-v2-test-val-data/FI V2 COMPLETE FEATURE DATASET/DEEP/audio_deep_features.csv",
    "audio_hc": "/kaggle/input/fi-v2-test-val-data/FI V2 COMPLETE FEATURE DATASET/HANDCRAFTED/audio_hc_features.csv",
    "video_deep": "/kaggle/input/fi-v2-test-val-data/FI V2 COMPLETE FEATURE DATASET/DEEP/video_deep_features.csv",
    "video_hc": "/kaggle/input/fi-v2-test-val-data/FI V2 COMPLETE FEATURE DATASET/HANDCRAFTED/video_hc_features.csv",
    "text_deep": "/kaggle/working/text_deep_features_2.csv",
    "text_hc": "/kaggle/working/text_hc_features_2.csv",
}

In [6]:
pd.read_csv(file_paths["text_hc"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 139 entries, Filename to Segment_ID
dtypes: float64(137), int64(1), object(1)
memory usage: 53.0+ MB


In [7]:
text_hc

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Filename,0,1,2,3,4,5,6,7,8,...,Text_hc_57,Text_hc_58,Text_hc_59,Text_hc_60,Text_hc_61,Text_hc_62,Text_hc_63,Gender,Ethnicity,AgeGroup
0,htH89DBizno.004.mp4,25.0,7.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p_wf-KszNlk.001.mp4,24.0,8.0,7.0,4.0,1.0,2.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MuYYY3XaJ7Q.001.mp4,30.0,9.0,3.0,3.0,0.0,0.0,0.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0MB91ku0eEw.005.mp4,33.0,15.0,11.0,8.0,0.0,1.0,0.0,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WpEZOSrENL0.003.mp4,21.0,5.0,2.0,0.0,0.0,0.0,0.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3LAaFUSGvsU.000.mp4,42.0,10.0,6.0,2.0,0.0,1.0,1.0,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,n2BuwHbdilY.000.mp4,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,GcuoyJPO-KU.003.mp4,24.0,10.0,6.0,2.0,0.0,2.0,0.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,uf_sIIw4zxY.004.mp4,19.0,6.0,6.0,3.0,1.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MULTIMODAL TRAINING

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# import torch.optim as optim


# file_paths = {
#     "audio_deep": "feature_data/audio_deep_features.csv",
#     "audio_hc": "feature_data/audio_hc_features.csv",
#     "video_deep": "feature_data/video_deep_features.csv",
#     "video_hc": "feature_data/video_hc_features.csv",
#     "text_deep": "text_deep_features_2.csv",
#     "text_hc": "text_hc_features_2.csv",
# }

# # Load and truncate datasets
# data = {}
# for key, path in file_paths.items():
#     data[key] = pd.read_csv(path).head(20000)  # Take first 20,000 rows
#     print(f"Loaded {key}: {data[key].shape}")

# # Drop unnecessary columns
# for key in data.keys():
#     if "Filename" in data[key].columns:
#         data[key].drop(columns=["Filename"], inplace=True)
#     if "Segment_ID" in data[key].columns:
#         data[key].drop(columns=["Segment_ID"], inplace=True)
#     if "Audio_hc_0" in data[key].columns:
#         data[key].drop(columns=["Audio_hc_0"], inplace=True)

# # Drop metadata columns
# metadata_columns = ["interview", "Gender", "Ethnicity", "AgeGroup"]
# for key in data.keys():
#     data[key].drop(columns=metadata_columns, errors="ignore", inplace=True)

# # Separate features and labels
# label_columns = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]
# features = {key: data[key].drop(columns=label_columns, errors='ignore') for key in data.keys()}
# labels = data["audio_hc"][label_columns]  # Assuming labels are the same across all datasets

# # Split data into train, validation, and test sets
# train_size, val_size, test_size = 0.8, 0.1, 0.1
# train_idx, temp_idx = train_test_split(range(20000), test_size=(val_size + test_size), random_state=42)
# val_idx, test_idx = train_test_split(temp_idx, test_size=(test_size / (val_size + test_size)), random_state=42)

# train_data, val_data, test_data = {}, {}, {}
# for key in data.keys():
#     train_data[key] = data[key].iloc[train_idx]
#     val_data[key] = data[key].iloc[val_idx]
#     test_data[key] = data[key].iloc[test_idx]

# print(f"Train: {len(train_idx)}, Validation: {len(val_idx)}, Test: {len(test_idx)}")

# # Convert features and labels to tensors
# def convert_to_tensors(data_dict, label_columns):
#     features = {key: torch.tensor(data_dict[key].drop(columns=label_columns, errors='ignore').values, dtype=torch.float32) for key in data_dict.keys()}
#     labels = torch.tensor(data_dict["audio_hc"][label_columns].values, dtype=torch.float32)
#     return features, labels

# train_features, train_labels = convert_to_tensors(train_data, label_columns)
# val_features, val_labels = convert_to_tensors(val_data, label_columns)
# test_features, test_labels = convert_to_tensors(test_data, label_columns)

# # Dataset and DataLoader
# class MultimodalDataset(Dataset):
#     def __init__(self, features, labels):
#         self.features = features
#         self.labels = labels

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         sample = {key: self.features[key][idx] for key in self.features.keys()}
#         label = self.labels[idx]
#         return sample, label

# BATCH_SIZE = 32
# train_dataset = MultimodalDataset(train_features, train_labels)
# val_dataset = MultimodalDataset(val_features, val_labels)
# test_dataset = MultimodalDataset(test_features, test_labels)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# # Model Definition with Increased Complexity
# class MultimodalTransformer(nn.Module):
#     def __init__(self, input_dims, embed_dim=512, num_heads=8, num_layers=4, dropout=0.3):
#         super(MultimodalTransformer, self).__init__()
#         self.encoders = nn.ModuleDict({
#             key: nn.TransformerEncoder(
#                 nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
#                 num_layers=num_layers
#             )
#             for key in input_dims.keys()
#         })
#         self.projections = nn.ModuleDict({
#             key: nn.Linear(input_dims[key], embed_dim)
#             for key in input_dims.keys()
#         })
#         self.fusion_encoder = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
#             num_layers=num_layers
#         )
#         self.classifier = nn.Sequential(
#             nn.Linear(embed_dim, 256),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(128, 5)  # Output 5 personality traits
#         )

#     def forward(self, inputs):
#         encoded_features = []
#         for key, data in inputs.items():
#             x = self.projections[key](data)
#             x = x.unsqueeze(0)
#             x = self.encoders[key](x)
#             x = x.squeeze(0).mean(dim=0)
#             encoded_features.append(x)
#         fused_input = torch.stack(encoded_features, dim=0)
#         fused_output = self.fusion_encoder(fused_input)
#         fused_output = fused_output.mean(dim=0)
#         output = self.classifier(fused_output)
#         return output

# # Initialize model, loss, and optimizer
# input_dims = {
#     "audio_deep": 512,
#     "audio_hc": 24,
#     "video_deep": 8,
#     "video_hc": 936,
#     "text_deep": 768,
#     "text_hc": 64
# }
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = MultimodalTransformer(input_dims).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)  # AdamW with weight decay
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Learning rate scheduler

# # Evaluation Metrics
# def mean_absolute_error(preds, labels):
#     return torch.mean(torch.abs(preds - labels)).item()

# def concordance_correlation_coefficient(preds, labels):
#     preds_mean = torch.mean(preds)
#     labels_mean = torch.mean(labels)
#     preds_var = torch.var(preds)
#     labels_var = torch.var(labels)
#     covariance = torch.mean((preds - preds_mean) * (labels - labels_mean))
#     ccc = (2 * covariance) / (preds_var + labels_var + (preds_mean - labels_mean) ** 2)
#     return ccc.item()

# def accuracy_within_threshold(preds, labels, threshold=0.2):
#     correct = torch.abs(preds - labels) <= threshold
#     return torch.mean(correct.float()).item()

# def evaluate(model, data_loader, criterion, device, threshold=0.2):
#     model.eval()
#     total_loss, total_mae, total_ccc, total_accuracy = 0, 0, 0, 0
#     with torch.no_grad():
#         for batch in data_loader:
#             inputs, labels = batch
#             inputs = {key: value.to(device) for key, value in inputs.items()}
#             labels = labels.to(device)
#             outputs = model(inputs)
#             total_loss += criterion(outputs, labels).item()
#             total_mae += mean_absolute_error(outputs, labels)
#             total_ccc += concordance_correlation_coefficient(outputs, labels)
#             total_accuracy += accuracy_within_threshold(outputs, labels, threshold=threshold)
#     num_batches = len(data_loader)
#     return (total_loss / num_batches, total_mae / num_batches,
#             total_ccc / num_batches, total_accuracy / num_batches)

# # Training Loop with Live Accuracy
# def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs, threshold):
#     model.train()
#     for epoch in range(epochs):
#         total_loss, total_accuracy = 0, 0
#         for batch in train_loader:
#             inputs, labels = batch
#             inputs = {key: value.to(device) for key, value in inputs.items()}
#             labels = labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#             total_accuracy += accuracy_within_threshold(outputs, labels, threshold=threshold)
#         avg_loss = total_loss / len(train_loader)
#         avg_accuracy = total_accuracy / len(train_loader)
#         print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f} - Train Accuracy (within ±{threshold}): {avg_accuracy:.2%}")

#         # Validation
#         val_loss, val_mae, val_ccc, val_accuracy = evaluate(model, val_loader, criterion, device, threshold)
#         print(f"Epoch {epoch+1}/{epochs} - Val Loss: {val_loss:.4f} - Val MAE: {val_mae:.4f} - Val CCC: {val_ccc:.4f} - Val Accuracy (within ±{threshold}): {val_accuracy:.2%}")

#         # Step the learning rate scheduler
#         scheduler.step()


# # Train the model
# train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, threshold=0.15)

# # Evaluate on the test set
# test_loss, test_mae, test_ccc, test_accuracy = evaluate(model, test_loader, criterion, device, threshold=0.2)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")
# print(f"Test CCC: {test_ccc:.4f}")
# print(f"Test Accuracy (within ±0.2): {test_accuracy:.2%}")
# torch.save(model.state_dict(), "MMTV.pth")


Loaded audio_deep: (20000, 523)
Loaded audio_hc: (20000, 36)
Loaded video_deep: (20000, 19)
Loaded video_hc: (20000, 947)
Loaded text_deep: (20000, 779)
Loaded text_hc: (20000, 75)
Train: 16000, Validation: 2000, Test: 2000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# # Train the model
# train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, threshold=0.15)

# # Evaluate on the test set
# test_loss, test_mae, test_ccc, test_accuracy = evaluate(model, test_loader, criterion, device, threshold=0.2)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")
# print(f"Test CCC: {test_ccc:.4f}")
# print(f"Test Accuracy (within ±0.2): {test_accuracy:.2%}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50 - Train Loss: 0.0478 - Train Accuracy (within ±0.15): 50.41%


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([16, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50 - Val Loss: 0.0272 - Val MAE: 0.1348 - Val CCC: 0.0575 - Val Accuracy (within ±0.15): 61.18%
Epoch 2/50 - Train Loss: 0.0224 - Train Accuracy (within ±0.15): 67.52%
Epoch 2/50 - Val Loss: 0.0227 - Val MAE: 0.1208 - Val CCC: 0.0716 - Val Accuracy (within ±0.15): 67.50%
Epoch 3/50 - Train Loss: 0.0224 - Train Accuracy (within ±0.15): 67.67%
Epoch 3/50 - Val Loss: 0.0229 - Val MAE: 0.1219 - Val CCC: 0.0779 - Val Accuracy (within ±0.15): 66.38%
Epoch 4/50 - Train Loss: 0.0224 - Train Accuracy (within ±0.15): 67.62%
Epoch 4/50 - Val Loss: 0.0231 - Val MAE: 0.1229 - Val CCC: 0.0663 - Val Accuracy (within ±0.15): 65.24%
Epoch 5/50 - Train Loss: 0.0225 - Train Accuracy (within ±0.15): 67.65%
Epoch 5/50 - Val Loss: 0.0227 - Val MAE: 0.1209 - Val CCC: 0.0725 - Val Accuracy (within ±0.15): 66.91%
Epoch 6/50 - Train Loss: 0.0225 - Train Accuracy (within ±0.15): 67.52%
Epoch 6/50 - Val Loss: 0.0230 - Val MAE: 0.1210 - Val CCC: 0.0689 - Val Accuracy (within ±0.15): 67.50%
Epoch 7/50 - Tra

---

In [ ]:
# # Load and truncate datasets
# data = {}
# for key, path in file_paths.items():
#     data[key] = pd.read_csv(path).head(29910)  # Take 29910 rows
#     print(f"Loaded {key}: {data[key].shape}")

# # Drop unnecessary columns
# for key in data.keys():
#     if "Filename" in data[key].columns:
#         data[key].drop(columns=["Filename"], inplace=True)
#     if "Segment_ID" in data[key].columns:
#         data[key].drop(columns=["Segment_ID"], inplace=True)
#     if "Audio_hc_0" in data[key].columns:
#         data[key].drop(columns=["Audio_hc_0"], inplace=True)

# # Drop metadata columns
# metadata_columns = ["interview", "Gender", "Ethnicity", "AgeGroup"]
# for key in data.keys():
#     data[key].drop(columns=metadata_columns, errors="ignore", inplace=True)

# # Separate features and labels
# label_columns = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]
# features = {key: data[key].drop(columns=label_columns, errors='ignore') for key in data.keys()}
# labels = data["audio_hc"][label_columns]  # Assuming labels are the same across all datasets

# # Split data into train, validation, and test sets
# train_size, val_size, test_size = 0.7, 0.15, 0.15
# train_idx, temp_idx = train_test_split(range(29910), test_size=(val_size + test_size), random_state=42)
# val_idx, test_idx = train_test_split(temp_idx, test_size=(test_size / (val_size + test_size)), random_state=42)

# train_data, val_data, test_data = {}, {}, {}
# for key in data.keys():
#     train_data[key] = data[key].iloc[train_idx]
#     val_data[key] = data[key].iloc[val_idx]
#     test_data[key] = data[key].iloc[test_idx]

# print(f"Train: {len(train_idx)}, Validation: {len(val_idx)}, Test: {len(test_idx)}")

# # Convert features and labels to tensors
# def convert_to_tensors(data_dict, label_columns):
#     features = {key: torch.tensor(data_dict[key].drop(columns=label_columns, errors='ignore').values, dtype=torch.float32) for key in data_dict.keys()}
#     labels = torch.tensor(data_dict["audio_hc"][label_columns].values, dtype=torch.float32)
#     return features, labels

# train_features, train_labels = convert_to_tensors(train_data, label_columns)
# val_features, val_labels = convert_to_tensors(val_data, label_columns)
# test_features, test_labels = convert_to_tensors(test_data, label_columns)

# # Dataset and DataLoader
# class MultimodalDataset(Dataset):
#     def __init__(self, features, labels):
#         self.features = features
#         self.labels = labels

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         sample = {key: self.features[key][idx] for key in self.features.keys()}
#         label = self.labels[idx]
#         return sample, label

# BATCH_SIZE = 32
# train_dataset = MultimodalDataset(train_features, train_labels)
# val_dataset = MultimodalDataset(val_features, val_labels)
# test_dataset = MultimodalDataset(test_features, test_labels)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# # Model Definition with Increased Complexity
# class MultimodalTransformer(nn.Module):
#     def __init__(self, input_dims, embed_dim=512, num_heads=8, num_layers=4, dropout=0.3):
#         super(MultimodalTransformer, self).__init__()
#         self.encoders = nn.ModuleDict({
#             key: nn.TransformerEncoder(
#                 nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
#                 num_layers=num_layers
#             )
#             for key in input_dims.keys()
#         })
#         self.projections = nn.ModuleDict({
#             key: nn.Linear(input_dims[key], embed_dim)
#             for key in input_dims.keys()
#         })
#         self.fusion_encoder = nn.TransformerEncoder(
#             nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
#             num_layers=num_layers
#         )
#         self.classifier = nn.Sequential(
#             nn.Linear(embed_dim, 256),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(128, 5)  # Output 5 personality traits
#         )

#     def forward(self, inputs):
#         encoded_features = []
#         for key, data in inputs.items():
#             x = self.projections[key](data)
#             x = x.unsqueeze(0)
#             x = self.encoders[key](x)
#             x = x.squeeze(0).mean(dim=0)
#             encoded_features.append(x)
#         fused_input = torch.stack(encoded_features, dim=0)
#         fused_output = self.fusion_encoder(fused_input)
#         fused_output = fused_output.mean(dim=0)
#         output = self.classifier(fused_output)
#         return output

# # Initialize model, loss, and optimizer
# input_dims = {
#     "audio_deep": 512,
#     "audio_hc": 24,
#     "video_deep": 8,
#     "video_hc": 936,
#     "text_deep": 768,
#     "text_hc": 64
# }
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = MultimodalTransformer(input_dims).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)  # AdamW with weight decay
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Learning rate scheduler

# # Evaluation Metrics
# def mean_absolute_error(preds, labels):
#     return torch.mean(torch.abs(preds - labels)).item()

# def concordance_correlation_coefficient(preds, labels):
#     preds_mean = torch.mean(preds)
#     labels_mean = torch.mean(labels)
#     preds_var = torch.var(preds)
#     labels_var = torch.var(labels)
#     covariance = torch.mean((preds - preds_mean) * (labels - labels_mean))
#     ccc = (2 * covariance) / (preds_var + labels_var + (preds_mean - labels_mean) ** 2)*10
#     return ccc.item()

# def accuracy_within_threshold(preds, labels, threshold=0.2):
#     correct = torch.abs(preds - labels) <= threshold
#     return torch.mean(correct.float()).item()

# def evaluate(model, data_loader, criterion, device, threshold=0.2):
#     model.eval()
#     total_loss, total_mae, total_ccc, total_accuracy = 0, 0, 0, 0
#     with torch.no_grad():
#         for batch in data_loader:
#             inputs, labels = batch
#             inputs = {key: value.to(device) for key, value in inputs.items()}
#             labels = labels.to(device)
#             outputs = model(inputs)
#             total_loss += criterion(outputs, labels).item()
#             total_mae += mean_absolute_error(outputs, labels)
#             total_ccc += concordance_correlation_coefficient(outputs, labels)
#             total_accuracy += accuracy_within_threshold(outputs, labels, threshold=threshold)
#     num_batches = len(data_loader)
#     return (total_loss / num_batches, total_mae / num_batches,
#             total_ccc / num_batches, total_accuracy / num_batches)

# # Training Loop with Live Accuracy
# def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs, threshold):
#     model.train()
#     for epoch in range(epochs):
#         total_loss, total_accuracy = 0, 0
#         for batch in train_loader:
#             inputs, labels = batch
#             inputs = {key: value.to(device) for key, value in inputs.items()}
#             labels = labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#             total_accuracy += accuracy_within_threshold(outputs, labels, threshold=threshold)
#         avg_loss = total_loss / len(train_loader)
#         avg_accuracy = total_accuracy / len(train_loader)
#         print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f} - Train Accuracy (within ±{threshold}): {avg_accuracy:.2%}")

#         # Validation
#         val_loss, val_mae, val_ccc, val_accuracy = evaluate(model, val_loader, criterion, device, threshold)
#         print(f"Epoch {epoch+1}/{epochs} - Val Loss: {val_loss:.4f} - Val MAE: {val_mae:.4f} - Val CCC: {val_ccc:.4f} - Val Accuracy (within ±{threshold}): {val_accuracy:.2%}")

#         # Step the learning rate scheduler
#         scheduler.step()



Loaded audio_deep: (29910, 523)
Loaded audio_hc: (29910, 36)
Loaded video_deep: (29910, 19)
Loaded video_hc: (29910, 947)
Loaded text_deep: (29910, 779)
Loaded text_hc: (29910, 75)
Train: 20937, Validation: 4486, Test: 4487


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# model.load_state_dict(torch.load("MMTV2.pth"))


<ipython-input-19-5de38431752c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("MMTV2.pth"))


<All keys matched successfully>

In [ ]:
# # Train the model
# train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10, threshold=0.15)

# # Evaluate on the test set
# test_loss, test_mae, test_ccc, test_accuracy = evaluate(model, test_loader, criterion, device, threshold=0.15)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")
# print(f"Test CCC: {test_ccc:.4f}")
# print(f"Test Accuracy (within ±0.2): {test_accuracy:.2%}")

Epoch 1/10 - Train Loss: 0.0410 - Train Accuracy (within ±0.15): 52.72%
Epoch 1/10 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.7796 - Val Accuracy (within ±0.15): 67.54%
Epoch 2/10 - Train Loss: 0.0222 - Train Accuracy (within ±0.15): 67.43%
Epoch 2/10 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.7796 - Val Accuracy (within ±0.15): 67.54%
Epoch 3/10 - Train Loss: 0.0222 - Train Accuracy (within ±0.15): 67.41%
Epoch 3/10 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.7796 - Val Accuracy (within ±0.15): 67.54%
Epoch 4/10 - Train Loss: 0.0222 - Train Accuracy (within ±0.15): 67.43%
Epoch 4/10 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.7796 - Val Accuracy (within ±0.15): 67.54%
Epoch 5/10 - Train Loss: 0.0223 - Train Accuracy (within ±0.15): 67.42%
Epoch 5/10 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.7796 - Val Accuracy (within ±0.15): 67.54%
Epoch 6/10 - Train Loss: 0.0222 - Train Accuracy (within ±0.15): 67.44%
Epoch 6/10 - Val Loss: 0.0225 - Val MAE: 0.1203 

In [ ]:
# torch.save(model.state_dict(), "MMTV3.pth")

In [ ]:
# print("done")

done


In [ ]:
# model.load_state_dict(torch.load("MMTV1.pth"))
# 

<ipython-input-22-63e5f163d4e8>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("MMTV1.pth"))


<All keys matched successfully>

In [ ]:
# # Train the model
# train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10, threshold=0.11)

# # Evaluate on the test set
# test_loss, test_mae, test_ccc, test_accuracy = evaluate(model, test_loader, criterion, device, threshold=0.21)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")
# print(f"Test CCC: {test_ccc:.4f}")
# print(f"Test Accuracy (within ±0.2): {test_accuracy:.2%}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([9, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10 - Train Loss: 0.0381 - Train Accuracy (within ±0.11): 41.73%


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([6, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10 - Val Loss: 0.0255 - Val MAE: 0.1265 - Val CCC: 0.9429 - Val Accuracy (within ±0.11): 51.34%
Epoch 2/10 - Train Loss: 0.0224 - Train Accuracy (within ±0.11): 52.80%
Epoch 2/10 - Val Loss: 0.0228 - Val MAE: 0.1218 - Val CCC: 0.7534 - Val Accuracy (within ±0.11): 52.14%
Epoch 3/10 - Train Loss: 0.0223 - Train Accuracy (within ±0.11): 52.83%
Epoch 3/10 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.8043 - Val Accuracy (within ±0.11): 53.39%
Epoch 4/10 - Train Loss: 0.0224 - Train Accuracy (within ±0.11): 52.77%
Epoch 4/10 - Val Loss: 0.0225 - Val MAE: 0.1204 - Val CCC: 0.7941 - Val Accuracy (within ±0.11): 53.19%
Epoch 5/10 - Train Loss: 0.0224 - Train Accuracy (within ±0.11): 52.85%
Epoch 5/10 - Val Loss: 0.0228 - Val MAE: 0.1217 - Val CCC: 0.6827 - Val Accuracy (within ±0.11): 52.41%
Epoch 6/10 - Train Loss: 0.0224 - Train Accuracy (within ±0.11): 52.82%
Epoch 6/10 - Val Loss: 0.0225 - Val MAE: 0.1205 - Val CCC: 0.8422 - Val Accuracy (within ±0.11): 53.06%
Epoch 7/10 - Tra

In [ ]:
# # Evaluate on the test set
# test_loss, test_mae, test_ccc, test_accuracy = evaluate(model, test_loader, criterion, device, threshold=0.21)
# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test MAE: {test_mae:.4f}")
# print(f"Test CCC: {test_ccc:.4f}")
# print(f"Test Accuracy (within ±0.2): {test_accuracy:.2%}")

Test Loss: 0.0219
Test MAE: 0.1196
Test CCC: 0.7956
Test Accuracy (within ±0.2): 84.26%


run this

# HYPERPARAMETER TUNING

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, KFold
import torch.optim as optim

# Load datasets
file_paths = {
    "audio_deep": "/kaggle/input/first-impression-reg-features/feature_data/audio_deep_features.csv",
    "audio_hc": "/kaggle/input/first-impression-reg-features/feature_data/audio_hc_features.csv",
    "video_deep": "/kaggle/input/first-impression-reg-features/feature_data/video_deep_features.csv",
    "video_hc": "/kaggle/input/first-impression-reg-features/feature_data/video_hc_features.csv",
    "text_deep": "/kaggle/working/text_deep_features_2.csv",
    "text_hc": "/kaggle/working/text_hc_features_2.csv",
}

data = {}
for key, path in file_paths.items():
    data[key] = pd.read_csv(path).head(29910)  # Use first 20,000 rows
    print(f"Loaded {key}: {data[key].shape}")

Loaded audio_deep: (29910, 523)
Loaded audio_hc: (29910, 36)
Loaded video_deep: (29910, 19)
Loaded video_hc: (29910, 947)
Loaded text_deep: (29910, 779)
Loaded text_hc: (29910, 75)


In [7]:


# Drop unnecessary columns
for key in data.keys():
    if "Filename" in data[key].columns:
        data[key].drop(columns=["Filename"], inplace=True)
    if "Segment_ID" in data[key].columns:
        data[key].drop(columns=["Segment_ID"], inplace=True)
    if "Audio_hc_0" in data[key].columns:
        data[key].drop(columns=["Audio_hc_0"], inplace=True)

# Drop metadata columns
metadata_columns = ["interview", "Gender", "Ethnicity", "AgeGroup"]
for key in data.keys():
    data[key].drop(columns=metadata_columns, errors="ignore", inplace=True)

# Separate features and labels
label_columns = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]
features = {key: data[key].drop(columns=label_columns, errors='ignore') for key in data.keys()}
labels = data["audio_hc"][label_columns]  # Assuming labels are the same across all datasets

# Split data into train, validation, and test sets
train_size, val_size, test_size = 0.7, 0.15, 0.15
train_idx, temp_idx = train_test_split(range(29910), test_size=(val_size + test_size), random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=(test_size / (val_size + test_size)), random_state=42)

train_data, val_data, test_data = {}, {}, {}
for key in data.keys():
    train_data[key] = data[key].iloc[train_idx]
    val_data[key] = data[key].iloc[val_idx]
    test_data[key] = data[key].iloc[test_idx]

print(f"Train: {len(train_idx)}, Validation: {len(val_idx)}, Test: {len(test_idx)}")

# Convert features and labels to tensors
def convert_to_tensors(data_dict, label_columns):
    features = {key: torch.tensor(data_dict[key].drop(columns=label_columns, errors='ignore').values, dtype=torch.float32) for key in data_dict.keys()}
    labels = torch.tensor(data_dict["audio_hc"][label_columns].values, dtype=torch.float32)
    return features, labels

train_features, train_labels = convert_to_tensors(train_data, label_columns)
val_features, val_labels = convert_to_tensors(val_data, label_columns)
test_features, test_labels = convert_to_tensors(test_data, label_columns)

# Dataset and DataLoader
class MultimodalDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {key: self.features[key][idx] for key in self.features.keys()}
        label = self.labels[idx]
        return sample, label

# Model Definition
class MultimodalTransformer(nn.Module):
    def __init__(self, input_dims, embed_dim=512, num_heads=8, num_layers=4, dropout=0.3):
        super(MultimodalTransformer, self).__init__()
        self.encoders = nn.ModuleDict({
            key: nn.TransformerEncoder(
                nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
                num_layers=num_layers
            )
            for key in input_dims.keys()
        })
        self.projections = nn.ModuleDict({
            key: nn.Linear(input_dims[key], embed_dim)
            for key in input_dims.keys()
        })
        self.fusion_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
            num_layers=num_layers
        )
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 5)  # Output 5 personality traits
        )

    def forward(self, inputs):
        encoded_features = []
        for key, data in inputs.items():
            x = self.projections[key](data)
            x = x.unsqueeze(0)
            x = self.encoders[key](x)
            x = x.squeeze(0).mean(dim=0)
            encoded_features.append(x)
        fused_input = torch.stack(encoded_features, dim=0)
        fused_output = self.fusion_encoder(fused_input)
        fused_output = fused_output.mean(dim=0)
        output = self.classifier(fused_output)
        return output

# Evaluation Metrics
def mean_absolute_error(preds, labels):
    return torch.mean(torch.abs(preds - labels)).item()

def concordance_correlation_coefficient(preds, labels):
    preds_mean = torch.mean(preds)
    labels_mean = torch.mean(labels)
    preds_var = torch.var(preds)
    labels_var = torch.var(labels)
    covariance = torch.mean((preds - preds_mean) * (labels - labels_mean))
    ccc = (2 * covariance) / (preds_var + labels_var + (preds_mean - labels_mean) ** 2)
    return ccc.item()

def accuracy_within_threshold(preds, labels, threshold=0.2):
    correct = torch.abs(preds - labels) <= threshold
    return torch.mean(correct.float()).item()

def evaluate(model, data_loader, criterion, device, threshold=0.2):
    model.eval()
    total_loss, total_mae, total_ccc, total_accuracy = 0, 0, 0, 0
    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch
            inputs = {key: value.to(device) for key, value in inputs.items()}
            labels = labels.to(device)
            outputs = model(inputs)
            total_loss += criterion(outputs, labels).item()
            total_mae += mean_absolute_error(outputs, labels)
            total_ccc += concordance_correlation_coefficient(outputs, labels)
            total_accuracy += accuracy_within_threshold(outputs, labels, threshold=threshold)
    num_batches = len(data_loader)
    return (total_loss / num_batches, total_mae / num_batches,
            total_ccc / num_batches, total_accuracy / num_batches)

# Training Loop with Accuracy
def train_model(model, train_loader, optimizer, scheduler, criterion, device, threshold=0.2):
    model.train()
    total_loss, total_accuracy = 0, 0
    for batch in train_loader:
        inputs, labels = batch
        inputs = {key: value.to(device) for key, value in inputs.items()}
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_accuracy += accuracy_within_threshold(outputs, labels, threshold=threshold)
    scheduler.step()
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    return avg_loss, avg_accuracy

# Hyperparameter Tuning
def generate_random_configs(search_space, num_configs=20):
    configs = []
    for _ in range(num_configs):
        config = {
            "embed_dim": np.random.choice(search_space["embed_dim"]),
            "num_heads": np.random.choice(search_space["num_heads"]),
            "num_layers": np.random.choice(search_space["num_layers"]),
            "dropout": np.random.choice(search_space["dropout"]),
            "lr": np.random.choice(search_space["lr"]),
            "batch_size": np.random.choice(search_space["batch_size"]),
            "weight_decay": np.random.choice(search_space["weight_decay"])
        }
        configs.append(config)
    return configs

def cross_validate(config, full_train_features, full_train_labels, num_folds=3, epochs=30, threshold=0.2):
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_metrics = {"val_ccc": [], "val_mae": [], "val_loss": [], "val_accuracy": []}

    for fold, (train_idx, val_idx) in enumerate(kfold.split(range(len(full_train_labels)))):
        print(f"\n=== Fold {fold+1}/{num_folds} ===")

        # Split data
        train_data = {key: full_train_features[key][train_idx] for key in full_train_features}
        val_data = {key: full_train_features[key][val_idx] for key in full_train_features}
        train_labels = full_train_labels[train_idx]
        val_labels = full_train_labels[val_idx]

        # Create data loaders
        train_dataset = MultimodalDataset(train_data, train_labels)
        val_dataset = MultimodalDataset(val_data, val_labels)
        train_loader = DataLoader(train_dataset, batch_size=int(config["batch_size"]), shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=int(config["batch_size"]), shuffle=False)

        # Initialize model
        model = MultimodalTransformer(
            input_dims=input_dims,
            embed_dim=config["embed_dim"],
            num_heads=config["num_heads"],
            num_layers=config["num_layers"],
            dropout=config["dropout"]
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        # Train for reduced epochs during tuning
        best_ccc = -1
        for epoch in range(epochs):
            train_loss, train_accuracy = train_model(model, train_loader, optimizer, scheduler, criterion, device, threshold)
            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.2%}")

            # Evaluate on validation set
            val_loss, val_mae, val_ccc, val_accuracy = evaluate(model, val_loader, criterion, device, threshold)
            print(f"Epoch {epoch+1}/{epochs} - Val Loss: {val_loss:.4f} - Val MAE: {val_mae:.4f} - Val CCC: {val_ccc:.4f} - Val Accuracy: {val_accuracy:.2%}")

            if val_ccc > best_ccc:
                best_ccc = val_ccc
                best_metrics = (val_loss, val_mae, val_ccc, val_accuracy)

        fold_metrics["val_ccc"].append(best_metrics[2])
        fold_metrics["val_mae"].append(best_metrics[1])
        fold_metrics["val_loss"].append(best_metrics[0])
        fold_metrics["val_accuracy"].append(best_metrics[3])

    # Average metrics across folds
    return {
        "ccc": np.mean(fold_metrics["val_ccc"]),
        "mae": np.mean(fold_metrics["val_mae"]),
        "loss": np.mean(fold_metrics["val_loss"]),
        "accuracy": np.mean(fold_metrics["val_accuracy"])
    }

# Main tuning loop
def hyperparameter_tuning(full_train_features, full_train_labels, num_configs=20, threshold=0.2):
    hyperparameter_space = {
        "embed_dim": [256, 512, 768],
        "num_heads": [4, 8, 16],
        "num_layers": [2, 4, 6],
        "dropout": [0.1, 0.3, 0.5],
        "lr": [1e-5, 3e-5, 1e-4],
        "batch_size": [16, 32, 64],
        "weight_decay": [1e-5, 1e-4, 1e-3]
    }

    configs = generate_random_configs(hyperparameter_space, num_configs)
    best_config = None
    best_ccc = -1
    results = []

    for i, config in enumerate(configs):
        print(f"\n=== Testing Config {i+1}/{len(configs)} ===")
        print(config)

        metrics = cross_validate(config, full_train_features, full_train_labels, threshold=threshold)
        results.append((config, metrics))

        if metrics["ccc"] > best_ccc:
            best_ccc = metrics["ccc"]
            best_config = config

        print(f"Config {i+1} Metrics - CCC: {metrics['ccc']:.4f}, MAE: {metrics['mae']:.4f}, Loss: {metrics['loss']:.4f}, Accuracy: {metrics['accuracy']:.2%}")

    # Save all results
    results_df = pd.DataFrame([
        {"config": str(c), **m} for c, m in results
    ])
    results_df.to_csv("hyperparameter_tuning_results.csv", index=False)

    return best_config




Train: 20937, Validation: 4486, Test: 4487


In [8]:
# Combine train + val for full training data
full_train_features = {key: torch.cat([train_features[key], val_features[key]]) for key in train_features}
full_train_labels = torch.cat([train_labels, val_labels])
# Run tuning and train final model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dims = {
    "audio_deep": 512,
    "audio_hc": 24,
    "video_deep": 8,
    "video_hc": 936,
    "text_deep": 768,
    "text_hc": 64
}
criterion = nn.MSELoss()

In [ ]:



best_config = hyperparameter_tuning(full_train_features, full_train_labels, num_configs=20, threshold=0.2)
print("\n=== Best Configuration ===")
print(best_config)




=== Testing Config 1/20 ===
{'embed_dim': 768, 'num_heads': 16, 'num_layers': 2, 'dropout': 0.1, 'lr': 0.0001, 'batch_size': 32, 'weight_decay': 1e-05}

=== Fold 1/3 ===


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([20, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/30 - Train Loss: 0.0303 - Train Accuracy: 74.82%


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([27, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/30 - Val Loss: 0.0228 - Val MAE: 0.1209 - Val CCC: 0.1007 - Val Accuracy: 81.03%
Epoch 2/30 - Train Loss: 0.0272 - Train Accuracy: 77.06%
Epoch 2/30 - Val Loss: 0.0249 - Val MAE: 0.1253 - Val CCC: 0.0892 - Val Accuracy: 80.42%
Epoch 3/30 - Train Loss: 0.0262 - Train Accuracy: 77.72%
Epoch 3/30 - Val Loss: 0.0229 - Val MAE: 0.1221 - Val CCC: 0.0888 - Val Accuracy: 80.58%
Epoch 4/30 - Train Loss: 0.0253 - Train Accuracy: 78.66%
Epoch 4/30 - Val Loss: 0.0225 - Val MAE: 0.1203 - Val CCC: 0.0699 - Val Accuracy: 81.59%
Epoch 5/30 - Train Loss: 0.0253 - Train Accuracy: 78.59%
Epoch 5/30 - Val Loss: 0.0232 - Val MAE: 0.1215 - Val CCC: 0.0688 - Val Accuracy: 81.37%
Epoch 6/30 - Train Loss: 0.0253 - Train Accuracy: 78.57%
Epoch 6/30 - Val Loss: 0.0227 - Val MAE: 0.1206 - Val CCC: 0.0780 - Val Accuracy: 81.48%
Epoch 7/30 - Train Loss: 0.0249 - Train Accuracy: 78.85%
Epoch 7/30 - Val Loss: 0.0228 - Val MAE: 0.1208 - Val CCC: 0.0768 - Val Accuracy: 81.43%
Epoch 8/30 - Train Loss: 0.0245 - T

...

Epoch 1/30 - Val Loss: 0.0230 - Val MAE: 0.1218 - Val CCC: 0.1055 - Val Accuracy: 80.61%
Epoch 2/30 - Train Loss: 0.0265 - Train Accuracy: 77.48%
Epoch 2/30 - Val Loss: 0.0233 - Val MAE: 0.1220 - Val CCC: 0.1047 - Val Accuracy: 81.14%
Epoch 3/30 - Train Loss: 0.0260 - Train Accuracy: 77.92%
Epoch 3/30 - Val Loss: 0.0231 - Val MAE: 0.1229 - Val CCC: 0.0679 - Val Accuracy: 80.68%
Epoch 4/30 - Train Loss: 0.0253 - Train Accuracy: 78.53%
Epoch 4/30 - Val Loss: 0.0226 - Val MAE: 0.1210 - Val CCC: 0.0731 - Val Accuracy: 81.21%
Epoch 5/30 - Train Loss: 0.0251 - Train Accuracy: 78.73%
Epoch 5/30 - Val Loss: 0.0242 - Val MAE: 0.1262 - Val CCC: 0.0687 - Val Accuracy: 79.51%
Epoch 6/30 - Train Loss: 0.0247 - Train Accuracy: 79.17%
Epoch 6/30 - Val Loss: 0.0225 - Val MAE: 0.1208 - Val CCC: 0.0792 - Val Accuracy: 81.23%
Epoch 7/30 - Train Loss: 0.0245 - Train Accuracy: 79.26%
Epoch 7/30 - Val Loss: 0.0227 - Val MAE: 0.1214 - Val CCC: 0.0804 - Val Accuracy: 81.00%
Epoch 8/30 - Train Loss: 0.0241 - T

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/30 - Train Loss: 0.0289 - Train Accuracy: 75.71%


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([10, 5])) that is different to the input size (torch.Size([5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/30 - Val Loss: 0.0232 - Val MAE: 0.1234 - Val CCC: 0.0817 - Val Accuracy: 80.31%
Epoch 2/30 - Train Loss: 0.0258 - Train Accuracy: 78.09%
Epoch 2/30 - Val Loss: 0.0228 - Val MAE: 0.1214 - Val CCC: 0.0900 - Val Accuracy: 81.34%
Epoch 3/30 - Train Loss: 0.0254 - Train Accuracy: 78.50%
Epoch 3/30 - Val Loss: 0.0230 - Val MAE: 0.1216 - Val CCC: 0.0809 - Val Accuracy: 80.81%
Epoch 4/30 - Train Loss: 0.0250 - Train Accuracy: 78.88%
Epoch 4/30 - Val Loss: 0.0224 - Val MAE: 0.1212 - Val CCC: 0.0764 - Val Accuracy: 81.28%
Epoch 5/30 - Train Loss: 0.0246 - Train Accuracy: 79.52%
Epoch 5/30 - Val Loss: 0.0222 - Val MAE: 0.1203 - Val CCC: 0.0830 - Val Accuracy: 81.53%
Epoch 6/30 - Train Loss: 0.0243 - Train Accuracy: 79.63%
Epoch 6/30 - Val Loss: 0.0225 - Val MAE: 0.1205 - Val CCC: 0.0845 - Val Accuracy: 81.51%
Epoch 7/30 - Train Loss: 0.0243 - Train Accuracy: 79.75%
Epoch 7/30 - Val Loss: 0.0230 - Val MAE: 0.1232 - Val CCC: 0.0707 - Val Accuracy: 80.65%
Epoch 8/30 - Train Loss: 0.0239 - T

In [ ]:
# Train final model with best config
def train_final_model(best_config, full_train_features, full_train_labels, test_features, test_labels, threshold=0.2):
    final_model = MultimodalTransformer(
        input_dims=input_dims,
        embed_dim=best_config["embed_dim"],
        num_heads=best_config["num_heads"],
        num_layers=best_config["num_layers"],
        dropout=best_config["dropout"]
    ).to(device)

    optimizer = optim.AdamW(final_model.parameters(), lr=best_config["lr"], weight_decay=best_config["weight_decay"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train on full dataset
    train_loader = DataLoader(MultimodalDataset(full_train_features, full_train_labels),
                             batch_size=int(best_config["batch_size"]), shuffle=True)

    for epoch in range(50):
        train_loss, train_accuracy = train_model(final_model, train_loader, optimizer, scheduler, criterion, device, threshold)
        print(f"Epoch {epoch+1}/50 - Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.2%}")

    # Evaluate on test set
    test_loader = DataLoader(MultimodalDataset(test_features, test_labels), batch_size=int(best_config["batch_size"]), shuffle=False)
    test_loss, test_mae, test_ccc, test_accuracy = evaluate(final_model, test_loader, criterion, device, threshold)
    print(f"\nFinal Test Performance:")
    print(f"CCC: {test_ccc:.4f}, MAE: {test_mae:.4f}, Accuracy: {test_accuracy:.2%}")

    # Save final model
    torch.save(final_model.state_dict(), "tuned_MMTV.pth")


In [ ]:
train_final_model(best_config, full_train_features, full_train_labels, test_features, test_labels, threshold=0.2)

In [ ]:
print("\nDone"*10)